In [ ]:
from nbdev import *

In [ ]:
#default_exp evaluation

# Evaluation

> evaluation of modelling results

In [ ]:
#export
from torchtools.core import *
from torchtools.data import *
from torchtools.models import *
from torchtools.datasets import *
from torchtools.augmentations import *
#from torchtools.datablock import *
from torchtools.dataloader import *
from torchtools.experiments import *
from torchtools.configs import *
from torchtools.evaluation import *

In [ ]:
#export
import pandas as pd
import numpy as np
from fastai.basics import *
from datetime import datetime, timedelta

### Configuration

In [ ]:
#provide some necessary configuration input
is_colab = True #use colab experiment results
is_class = False #regression
#df_base path
# df__base_path = Path('./data/custom/bi_sample_anon.csv')
# df__base_path = Path('./data/custom/bi_sample_pruned_anon.csv')
df_base_path = Path('~/coding/python/betting/experiments/datasets/bi_sample_pruned_anon.csv').expanduser()
# df_base_path = Path('~/coding/python/betting/experiments/datasets/bi_sample_pruned_anon_222.csv').expanduser()
# df_base_path = Path('~/coding/python/betlib/data/processed/ts_experiments_anon_ts_exp_2020818.csv').expanduser()
# df_base_path = Path('~/coding/python/betting/experiments/datasets/bi_50_218_anon.csv').expanduser()

In [ ]:
eval_conf = EvalConfig(is_colab, is_class, df_base_path)

In [ ]:
eval_conf.df_base_path

Path('/home/johannes/coding/python/betting/experiments/datasets/bi_sample_pruned_anon.csv')

## Evaluation

- get best hyperparameters
- evaluate ensembling

### evaluate directly over specific experiments (identified by row index in df_results)

promising regreesion ensembles:
- [455, 559,535, 479, 187], solid but not spectacular over both valid and test sets, both test and valid hc around 2.7% in the 0.9 quantile

In [ ]:
idxs = [455,559,535,479,187]

In [ ]:
basic_eval(eval_conf, idxs, quantile=0.95)

                y0           y1
mean      4.549413     1.480021
sum    5686.765810  1850.025918
count  1250.000000  1250.000000
single results [array([1.87766792]), array([1.4628895]), array([2.64322925]), array([-3.33851048]), array([1.36463485])]
mean result 0.8019822081599994


In [ ]:
basic_eval(eval_conf, idxs, test=True, quantile=0.95)

                y0           y1
mean      2.843560     2.558725
sum    3554.450608  3198.406529
count  1250.000000  1250.000000
single results [array([0.12496947]), array([-1.34115441]), array([1.98523216]), array([3.61429448]), array([3.45998786])]
mean result 1.56866591056


### Filtering

filter df_results and extract indices, e.g. according to some sorting
- make sure you filter at least for 'ds', otherwise the experiments might not be comaparable

Some observations:
- test results much better for seed 1234
- test results better for 2048 batch size, compare [477, 478, 479]
- test results sensitive to quantile

In [ ]:
filt = {'seed':[1234], 'magnitude': [0.4], 'ds':['bi_sample_pruned_anon'], 'n_epochs':[20], 
       'target':['1yhc'], 'col_config':['10sl_4c_2d'], 'N':[4]}

In [ ]:
filt={'n_epochs': 20,
 'max_lr': 0.0001,
 'wd': 0.03,
 'pct_start': 0.3,
 'div_factor': 25, 'ds':['bi_sample_pruned_anon'], 'target':['1yhc'], 'col_config':['10sl_4c_2d']}

In [ ]:
filt_df(eval_conf.df_results, filt).groupby(fitting_params)[eval_conf.m_cols].agg(['mean', 'count'])

unweighted_profit_0_value        \
                                                               mean count   
n_epochs max_lr wd   pct_start div_factor                                   
20       0.0001 0.03 0.3       25.0                        0.128621    11   

                                          unweighted_profit_05_1_value        
                                                                  mean count  
n_epochs max_lr wd   pct_start div_factor                                     
20       0.0001 0.03 0.3       25.0                           0.130237    11

In [ ]:
top_idxs = get_top(eval_conf.df_results, filt, by=loss_cols[0], n=3)

In [ ]:
top_idxs

[682, 681, 684]

In [ ]:
basic_eval(eval_conf, top_idxs, quantile=0.9)

                y0           y1
mean      3.259326     1.288097
sum    8148.315255  3220.241416
count  2500.000000  2500.000000
single results [array([0.16035809]), array([2.25463038]), array([1.09268488])]
mean result 1.169224447333333


In [ ]:
basic_eval(eval_conf, top_idxs, test=True, quantile=0.9)

                y0           y1
mean      1.424431     1.754542
sum    3561.078670  4386.353988
count  2500.000000  2500.000000
single results [array([0.79572659]), array([2.35773731]), array([-1.33820847])]
mean result 0.6050851422666664


### Comparative Evaluation

- sort by metrics (which metric?) or loss
- which ensembles perform bets?

### Use Case #1

We start with a rough filter, e.g. all runs with a certain column configuration, or between certain dates


In [ ]:
df_path

Path('data/custom/bi_sample_anon.csv')

In [ ]:
is_colab = True
is_class = False
df_base_path =  Path('~/coding/python/betting/experiments/datasets/bi_sample_pruned_anon.csv').expanduser()

In [ ]:
eval_conf = EvalConfig(is_colab, is_class, df_base_path)

In [ ]:
## basic filter
filt = {
 'ds': ['bi_sample_pruned_anon'],
 'target': ['1yhc'],
 'col_config': ['10sl_4c_2d'],
    'bs':[1024, 2048]
}

# filt = {'ds': ['bi_sample_pruned_anon'], 'target': ['1yhc'], 'col_config': ['10sl_4c_2d'],
#         'start_date':datetime(2020,10,5)}


Get a rough overview over the results with respect to certain parameters

In [ ]:
get_best_params??

In [ ]:
get_best_params(eval_conf, filt, by=None)

,n_epochs,max_lr,wd,pct_start,div_factor,unweighted_profit_0_value
14,20,0.00001,0.30,0.3,25.0,-0.008061
11,15,0.00030,0.30,0.3,25.0,0.122269
9,15,0.00030,0.03,0.3,25.0,0.124733
10,15,0.00030,0.10,0.3,25.0,0.126022
20,20,0.00010,0.30,0.3,25.0,0.135028
21,20,0.00030,0.03,0.3,25.0,0.141151
18,20,0.00010,0.03,0.3,25.0,0.160540
8,15,0.00010,0.30,0.3,25.0,0.167235
23,20,0.00030,0.30,0.3,25.0,0.170094
6,15,0.00010,0.03,0.3,25.0,0.181716


In [ ]:
params = ['n_epochs', 'max_lr', 'wd', 'N', 'magnitude', 'bs', ['n_epochs', 'max_lr']]

`rough_param_comparison` can easily lead to wrong conclusions, always compare with `get_best_params` to get a better picture!!

In [ ]:
rough_param_comparison(eval_conf, filt, by=eval_conf.m_cols[0], params=params)

n_epochs
          unweighted_profit_0_value
n_epochs                           
15                         0.219773
20                         0.153399
max_lr
         unweighted_profit_0_value
max_lr                            
0.00003                   0.226364
0.00030                   0.158724
0.00010                   0.144726
0.00001                   0.098214
wd
      unweighted_profit_0_value
wd                             
0.10                   0.245226
0.03                   0.227855
0.30                   0.143134
N
   unweighted_profit_0_value
N                           
3                   0.274956
4                   0.141574
magnitude
           unweighted_profit_0_value
magnitude                           
0.4                         0.165112
bs
      unweighted_profit_0_value
bs                             
1024                   0.242049
2048                   0.151400
['n_epochs', 'max_lr']
                  unweighted_profit_0_value
n_epochs max_lr               

In [ ]:
## do the same thing with respect to the loss function
rough_param_comparison(eval_conf, filt, by=loss_cols[0], params=params)

n_epochs
          val_loss
n_epochs          
15       -0.995926
20       -0.987428
max_lr
         val_loss
max_lr           
0.00030 -1.186649
0.00010 -1.139474
0.00003 -0.987880
0.00001 -0.693176
wd
      val_loss
wd            
0.10 -1.030925
0.03 -1.018370
0.30 -0.977937
N
   val_loss
N          
3 -1.034089
4 -0.979250
magnitude
           val_loss
magnitude          
0.4       -0.988928
bs
      val_loss
bs            
1024 -1.026067
2048 -0.982309
['n_epochs', 'max_lr']
                  val_loss
n_epochs max_lr           
20       0.00030 -1.192605
15       0.00010 -1.182821
         0.00030 -1.180693
20       0.00010 -1.132539
15       0.00003 -1.011400
20       0.00003 -0.984117
15       0.00001 -0.701174
20       0.00001 -0.690776


The rough comparison data can be used to refine the filter

In [ ]:
filt

{'ds': ['bi_sample_pruned_anon'],
 'target': ['1yhc'],
 'col_config': ['10sl_4c_2d'],
 'bs': [1024, 2048]}

In [ ]:
filt.update({'n_epochs':[15, 20], 'max_lr':[0.00003, 0.00001], 'wd':[0.1, 0.03]})

In [ ]:
filt

{'ds': ['bi_sample_pruned_anon'],
 'target': ['1yhc'],
 'col_config': ['10sl_4c_2d'],
 'bs': [1024, 2048],
 'n_epochs': [15, 20],
 'max_lr': [3e-05, 1e-05],
 'wd': [0.1, 0.03]}

now extract the top models with the refined filter

In [ ]:
top_idxs = get_top(eval_conf.df_results, filt, loss_cols[0], n=5)

In [ ]:
top_idxs = get_top(eval_conf.df_results, filt, by=eval_conf.m_cols[1], n=5)

In [ ]:
top_idxs

[186, 187, 223, 180, 222]

In [ ]:
basic_eval(eval_conf, top_idxs, quantile=0.9)

                y0           y1
mean      3.882994     2.315580
sum    9707.485467  5788.950586
count  2500.000000  2500.000000
single results [array([3.08517492]), array([2.90818978]), array([1.96469689]), array([2.67613986]), array([2.01273574])]
mean result 2.5293874374399996


In [ ]:
basic_eval(eval_conf, top_idxs, test=True, quantile=0.9)

                y0           y1
mean      2.053274     1.732733
sum    5133.184821  4331.832092
count  2500.000000  2500.000000
single results [array([2.70763156]), array([2.62903466]), array([0.54474034]), array([1.362647]), array([0.77373205])]
mean result 1.6035571224000003


In [ ]:
get_top??

In [ ]:
eval_conf.df_results.iloc[top_idxs][eval_conf.m_cols+fitting_params+augment_params+loss_cols+['seed','bs']+experiment_cols]

,unweighted_profit_0_value,unweighted_profit_05_1_value,n_epochs,max_lr,wd,pct_start,div_factor,N,magnitude,val_loss,trn_loss,seed,bs,arch,bs,ds_id
768,0.158891,0.159246,20,0.001,0.1,0.3,25.0,4,0.4,-1.212309,-0.991275,982,5096,InceptionTime,5096,bi_sample_pruned_anon_anon_10sl_4c_2d_1yhc_160_185_210
767,0.029969,0.044090,20,0.001,0.1,0.3,25.0,4,0.4,-1.177437,-1.070131,543,5096,InceptionTime,5096,bi_sample_pruned_anon_anon_10sl_4c_2d_1yhc_160_185_210
765,0.143286,0.078814,20,0.001,0.1,0.3,25.0,4,0.4,-1.175713,-1.070693,13,5096,InceptionTime,5096,bi_sample_pruned_anon_anon_10sl_4c_2d_1yhc_160_185_210
770,0.161165,0.023989,20,0.001,0.1,0.3,25.0,4,0.4,-1.145782,-0.989210,654321,5096,InceptionTime,5096,bi_sample_pruned_anon_anon_10sl_4c_2d_1yhc_160_185_210
769,0.056304,0.098999,20,0.001,0.1,0.3,25.0,4,0.4,-1.129718,-0.983229,12345,5096,InceptionTime,5096,bi_sample_pruned_anon_anon_10sl_4c_2d_1yhc_160_185_210
764,0.048759,0.018869,20,0.001,0.1,0.3,25.0,4,0.4,-1.084390,-1.025485,1234,5096,InceptionTime,5096,bi_sample_pruned_anon_anon_10sl_4c_2d_1yhc_160_185_210
766,-0.004369,-0.013245,20,0.001,0.1,0.3,25.0,4,0.4,-0.972975,-0.962063,15,5096,InceptionTime,5096,bi_sample_pruned_anon_anon_10sl_4c_2d_1yhc_160_185_210


In [ ]:
filt_df(eval_conf.df_results, filt)[eval_conf.m_cols+loss_cols]

,unweighted_profit_0_value,unweighted_profit_05_1_value,val_loss,trn_loss
722,0.032561,0.165483,-1.124519,-1.007780
723,0.141726,0.153839,-1.189306,-1.019087
724,0.096901,0.018509,-1.051002,-1.020670
725,0.172587,0.026612,-1.145170,-1.027829
726,0.142595,0.166711,-1.145099,-0.991317
727,0.163247,0.104556,-1.105765,-0.978222
728,0.285902,0.242673,-1.349013,-0.984784
736,0.108834,0.070548,-1.115453,-1.028581
737,0.179263,0.194968,-1.244646,-1.045480
738,0.023998,-0.008640,-0.998857,-1.011510


In [ ]:
get_top??